In [2]:
import medmnist
from medmnist import INFO, Evaluator

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import numpy as np

In [6]:
DATASET = "bloodmnist"
NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

dataset_info = INFO[DATASET]
task = dataset_info['task']
num_channels = dataset_info['n_channels']
num_classes = len(dataset_info['label'])

DatasetClass = getattr(medmnist, dataset_info['python_class'])

task: multi-class
num_channels: 3
num_classes: 8


In [8]:
data_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(mean=[.5], std=[0.5])
])

Compose(
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
)
